In [617]:
import pandas as pd
import re
import numpy as np

In [618]:
file_id = "1lfH64MX8NHuxn7745leZ6LaXRVLAAer77J336ZFOTIk"
gid = "1900938527"  # onglet cible
url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid={gid}"

df = pd.read_csv(url)  # éventuellement: encoding="utf-8"
df_clean = pd.DataFrame()
print(df.shape)


(46, 30)


In [619]:
df_clean = df

In [620]:
df_clean.columns

Index(['Horodateur', 'Quel est votre nationalité?',
       '  Dans quel pays résidez-vous actuellement ?  ',
       'Quelle est votre tranche d’âge ?  ',
       'Quelle est votre situation familiale ? ',
       'Quelle est votre tranche de revenus mensuels nets du foyer ? ',
       'À quelle fréquence voyagez vous à l’étranger (hors Europe) ?  ',
       'Avez-vous déjà voyagé au Japon ?  ',
       'Quelle durée de séjour avez-vous prévue ?  ',
       'Quelles régions du Japon vous intéressent le plus ? (Choisissez 3 max.)  ',
       'À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Découverte de la culture et de l’histoire (temples, traditions, samouraïs, geishas, etc.)]',
       'À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Gastronomie japonaise (sushis, ramen, wagyu, street food, etc.)]',
       'À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Paysages naturels et randonnées (monta

In [621]:
df_clean = df_clean.rename(columns={
    "Quel est votre nationalité?": "nationality",
    "  Dans quel pays résidez-vous actuellement ?  ": "country",
    "Quelle est votre tranche d’âge ?  ": "age_group",
    "Quelle est votre situation familiale ? ": "family_situation",
    "Quelle est votre tranche de revenus mensuels nets du foyer ? ": "household_income_in_€",
    "À quelle fréquence voyagez vous à l’étranger (hors Europe) ?  ": "travel_frequency",
    "Avez-vous déjà voyagé au Japon ?  ": "been_to_Japan",
    "Quelle durée de séjour avez-vous prévue ?  ": "Japan_vac_duration",
    "Quelles régions du Japon vous intéressent le plus ? (Choisissez 3 max.)  ": "most_wanted_pref_to_visit",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Découverte de la culture et de l’histoire (temples, traditions, samouraïs, geishas, etc.)]": "rating_interest_culture_and_history",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Gastronomie japonaise (sushis, ramen, wagyu, street food, etc.)]": "rating_interest_food",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Paysages naturels et randonnées (montagnes, volcans, cerisiers en fleurs, etc.)]": "rating_interest_nature_hiking",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Technologie, innovation et shopping (Tokyo high-tech, Akihabara, mode, etc.)]": "rating_interest_shopping_and_techno",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Festivals et événements (matsuri, concerts, sport, sumo, etc.)]": "rating_interest_events_and_festivals",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Bien-être (onsen, ryokan, détente)]": "rating_interest_wellness",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Parc d'attraction (Disneyland, Universal...)]": "rating_interest_theme_park",
    "Quel budget global prévoyez vous pour un voyage au Japon (par personne et par semaine , hors vol international ) ?  ": "Japan_budget_per_week",
    "Parmi les types d’hébergement suivants, lequel correspond le mieux à vos préférences principales pour un séjour au Japon ? ": "Japan_prefered_accomodation",
    "Quels sont les principaux freins ou difficultés que vous rencontrez (ou pourriez rencontrer) lors d’un voyage au Japon ? (Choisissez 3 max.)  ": "Japan_most_difficulties",
    "Si vous ne pouviez pas voyager au Japon, quelle destination alternative choisiriez-vous ?": "alternative_destination",
    "Quelle a été la principale raison pour laquelle vous auriez choisi cette destination plutôt que le Japon ? ": "alt_dest_main_reason",
    "Parmi les types d’hébergement suivants, lequel correspond le mieux à vos préférences principales lors de vos voyages dans d’autres pays (hors Japon)   ? ": "alt_dest_prefered_accomodation",
    "Lors de vos voyages dans d’autres pays (hors Japon), quel est votre budget moyen par semaine et par personne , hors vol international ?  ": "alt_dest_budget_per_week",
    "Lors de vos voyages dans d’autres pays (hors Japon), quel(s) mode(s) de transport utilisez-vous le plus souvent ?": "alt_dest_transportation",
    "Comment préparez-vous vos voyages en général ? (Multiple choix possible)": "trip_prep",
    "Quel canal utilisez-vous le plus pour réserver vos voyages ?  ": "booking_trip_channel",
    "Parmi les éléments suivants, lequel influence le plus votre choix de destination de vacances (hors Japon) ? ": "most_influencial_reason_to_choose_dest",
    "Lorsque vous voyagez en dehors du Japon, quelles sont les principales difficultés que vous rencontrez habituellement ?\n(Choisissez jusqu’à 3 réponses)": "alt_dest_most_difficulties",
    "Qu’est-ce qui rendrait le Japon plus attractif comme destination pour vous ?  ": "recomendation_to_improve_attractiveness"
})

In [622]:
def normalize_text(s):
    if pd.isna(s): return s
    return (str(s)
            .strip()
            .lower()
            .replace("é","e").replace("è", "e").replace("ê", "e")
            .replace("à","a").replace("ï", "i").replace("î", "i")
            .replace("$","").replace("€", ""))

mapping = {
    #France
    "france": "France",
    "française": "France",
    "français": "France",
    "francais": "France",
    "francaise": "France",
    "french": "France",
    "法国": "France",

    #China
    "chine": "China",
    "chinoise": "China",
    "chinois": "China",
    "中国": "China",
    "chinese": "China",

    #Taiwan
    "taïwan ": "Taiwan",
    "taiwan": "Taiwan",
    "taiwanais": "Taiwan",
    "taiwanaise": "Taiwan",

    #Vietnam
    "vietnam": "Vietnam",
    "vietnamien": "Vietnam",
    "vietnamienne": "Vietnam",

    #Portugal
    "portuguese": "Portugal",

    #Israel
    "israel": "Israel",
    "israël": "Israel",
    "israelien": "Israel",
    "israelienne": "Israel",
    "israélien": "Israel",

    #Spain
    "espagne": "Spain",

    #Germany
    "allemagne": "Germany",

}

if "nationality" in df_clean.columns:
    df_clean["nationality"] = (df_clean["nationality"]
                               .map(normalize_text)
                               .map(mapping)
                               .fillna(df_clean["nationality"]))
    
if "country" in df_clean.columns:
    df_clean["country"] = (df_clean["country"]
                               .map(normalize_text)
                               .map(mapping)
                               .fillna(df_clean["country"]))

In [623]:
df_clean.columns

Index(['Horodateur', 'nationality', 'country', 'age_group', 'family_situation',
       'household_income_in_€', 'travel_frequency', 'been_to_Japan',
       'Japan_vac_duration', 'most_wanted_pref_to_visit',
       'rating_interest_culture_and_history', 'rating_interest_food',
       'rating_interest_nature_hiking', 'rating_interest_shopping_and_techno',
       'rating_interest_events_and_festivals', 'rating_interest_wellness',
       'rating_interest_theme_park', 'Japan_budget_per_week',
       'Japan_prefered_accomodation', 'Japan_most_difficulties',
       'alternative_destination', 'alt_dest_main_reason',
       'alt_dest_prefered_accomodation', 'alt_dest_budget_per_week',
       'alt_dest_transportation', 'trip_prep', 'booking_trip_channel',
       'most_influencial_reason_to_choose_dest', 'alt_dest_most_difficulties',
       'recomendation_to_improve_attractiveness'],
      dtype='object')

In [624]:
print(df_clean["nationality"].value_counts())
print(df_clean["country"].value_counts())

nationality
France      32
China       10
Vietnam      1
Taiwan       1
Israel       1
Portugal     1
Name: count, dtype: int64
country
France      42
Spain        2
Germany      1
Portugal     1
Name: count, dtype: int64


In [625]:
print(df_clean["age_group"].value_counts())

age_group
25 – 34 ans        16
35 – 44 ans        13
45 – 54 ans         9
18 – 24 ans         3
55 – 64 ans         2
35–44 years old     2
18–24 years old     1
Name: count, dtype: int64


In [626]:
def clean_age (age):
    if pd.isna(age):
        return None
    age = str(age).strip()
    if age.startswith("18"):
        return "18-24"
    elif age.startswith("25"):
        return "25-34"
    elif age.startswith("35"):
        return "35-44"
    elif age.startswith("45"):
        return "45-54"
    elif age.startswith("55"):
        return "55-64"
    elif age.startswith("65"):
        return "65 and over"
    else:
        return "18 and less"

df_clean["age_group"] = df_clean["age_group"].apply(clean_age)


In [627]:
print(df_clean["age_group"].value_counts())

age_group
25-34    16
35-44    15
45-54     9
18-24     4
55-64     2
Name: count, dtype: int64


In [628]:
print(df_clean["family_situation"].value_counts())

family_situation
Célibataire                                        15
Marié(e)/Pacsé(e) avec enfant(s)                    9
En couple avec enfant(s)                            7
Marié(e)/Pacsé(e) sans enfant                       6
En couple sans enfant                               5
Préfère ne pas répondre                             1
Married / in a civil partnership, no children       1
Married / in a civil partnership, with children     1
In a relationship, no children                      1
Name: count, dtype: int64


In [629]:
family_situation_map = {
    
    "single": "Single",
    "celibataire": "Single",

    "en couple sans enfant": "Relationship_no_kids",
    "in a relationship, no children": "Relationship_no_kids",

    "en couple avec enfant(s)": "Relationship_with_kids",
    "in a relationship, with children": "Relationship_with_kids",

    "marie(e)/pacse(e) sans enfant": "Married_no_kids",
    "married / in a civil partnership, no children": "Married_no_kids",

    "marie(e)/pacse(e) avec enfant(s)": "Married_with_kids",
    "married / in a civil partnership, with children": "Married_with_kids",

    "prefere ne pas repondre": "Unknown",
    "prefer not to say": "Unknown"
}

df_clean["family_situation"] = (df_clean["family_situation"]
                            .map(normalize_text)
                            .map(family_situation_map)
                            .fillna(df_clean["family_situation"]))


In [630]:
print(df_clean["family_situation"].value_counts())

family_situation
Single                    15
Married_with_kids         10
Married_no_kids            7
Relationship_with_kids     7
Relationship_no_kids       6
Unknown                    1
Name: count, dtype: int64


In [631]:
print(df_clean["household_income_in_€"].value_counts())

household_income_in_€
1 500 – 1 999 €                       10
3 000 – 3 999 €                        6
2 000 – 2 499 €                        5
2 500 – 2 999 €                        5
5000 – 5 999 €                         5
Supérieur à 7 000 €                    3
Préfère ne pas répondre                3
Moins de 1 500 €                       3
4 000 – 4 999 €                        2
6 000 – 6 999 €                        1
$1,700 – $2,200 (~€1,500 – €1,999)     1
$3,400 – $4,400 (~€3,000 – €3,999)     1
Prefer not to say                      1
Name: count, dtype: int64


In [632]:
clean_income = {
    
    "moins de 1 500 ": "1500 and less",
    "less than 1,700 (~1,500)": "1500 and less",

    "1 500 – 1 999 ": "1500-1999",
    "1,700 – 2,200 (~1,500 – 1,999)": "1500-1999",

    "2 000 – 2 499 ": "2000-2499",
    "2,200 – 2,700 (~2,000 – 2,499)": "2000-2499",

    "2 500 – 2 999 ": "2500-2999",
    "2,800 – 3,300 (~2,500 – 2,999)": "2500-2999",

    "3 000 – 3 999 ": "3000-3999",
    "3,400 – 4,400 (~3,000 – 3,999)": "3000-3999",

    "4 000 – 4 999 ": "4000–4999",
    "4,500 – 5,500 (~4,000 – 4,999)": "4000–4999",

    "5000 – 5 999 ": "5000–5999",
    "5,600 – 6,600 (~5,000 – 5,999)": "5000–5999",

    "6 000 – 6 999 ": "6000–6999",
    "6,700 – 7,700 (~6,000 – 6,999)": "6000–6999",

    "superieur a 7 000 ": "7000 and more",
    "more than 7,800 (~7,000+)": "7000 and more",

    "prefere ne pas repondre": "Unknown",
    "prefer not to say": "Unknown",
    }

df_clean["household_income_in_€"] = (df_clean["household_income_in_€"]
                            .map(normalize_text)
                            .map(clean_income)
                            .fillna(df_clean["household_income_in_€"]))


In [633]:
print(df_clean["household_income_in_€"].value_counts())

household_income_in_€
1500-1999        11
3000-3999         7
2000-2499         5
2500-2999         5
5000–5999         5
Unknown           4
7000 and more     3
1500 and less     3
4000–4999         2
6000–6999         1
Name: count, dtype: int64


In [634]:
print(df_clean["travel_frequency"].value_counts())

travel_frequency
Tous les 2–3 ans                   14
Une fois par an                    14
Plusieurs fois par an               6
Une fois tous les 5 ans ou plus     5
Jamais                              4
Every 2–3 years                     2
Once every 5 years or more          1
Name: count, dtype: int64


In [635]:
clean_travel_frequency = {
    
    "jamais": "Never",
    "une fois tous les 5 ans ou plus": "Once every 5 years or more",
    "tous les 2–3 ans": "Every 2–3 years",
    "une fois par an": "Once a year",
    "plusieurs fois par an": "Several times a year",
    }

df_clean["travel_frequency"] = (df_clean["travel_frequency"]
                            .map(normalize_text)
                            .map(clean_travel_frequency)
                            .fillna(df_clean["travel_frequency"]))


In [636]:
print(df_clean["travel_frequency"].value_counts())

travel_frequency
Every 2–3 years               16
Once a year                   14
Several times a year           6
Once every 5 years or more     6
Never                          4
Name: count, dtype: int64


In [637]:
print(df_clean["been_to_Japan"].value_counts())

been_to_Japan
Non, mais j’aimerais y aller        29
Non, et je ne suis pas intéressé     7
Oui, une fois                        5
Oui, plusieurs fois                  2
No, but I would like to go           2
Yes, several times                   1
Name: count, dtype: int64


In [638]:
clean_been_to_japan = {

    "oui, une fois": "Yes, once",
    "oui, plusieurs fois": "Yes, several times",
    "non, mais j’aimerais y aller": "No, but I would like to go",
    "non, et je ne suis pas interesse": "No, and I’m not interested",
    }

df_clean["been_to_Japan"] = (df_clean["been_to_Japan"]
                            .map(normalize_text)
                            .map(clean_been_to_japan)
                            .fillna(df_clean["been_to_Japan"]))

print(df_clean["been_to_Japan"].value_counts())

been_to_Japan
No, but I would like to go    31
No, and I’m not interested     7
Yes, once                      5
Yes, several times             3
Name: count, dtype: int64


In [639]:
print(df_clean["Japan_vac_duration"].value_counts())

Japan_vac_duration
2 semaines                              15
3 semaines                               8
1 semaine                                5
Je ne sais pas / Pas assez renseigné     5
4 semaines                               2
2 weeks                                  2
Plus de 4 semaines                       1
More than 4 weeks                        1
Name: count, dtype: int64


In [640]:
clean_Japan_vac_duration = {
    
    "1 semaine": "1 week",
    "2 semaines": "2 weeks",
    "3 semaines": "3 weeks",
    "4 semaines": "4 weeks",
    "plus de 4 semaines": "More than 4 weeks",
    "je ne sais pas / pas assez renseigne": "I don’t know yet / Not sure"
    }

df_clean["Japan_vac_duration"] = (df_clean["Japan_vac_duration"]
                            .map(normalize_text)
                            .map(clean_Japan_vac_duration)
                            .fillna(df_clean["Japan_vac_duration"]))

print(df_clean["Japan_vac_duration"].value_counts())

Japan_vac_duration
2 weeks                        17
3 weeks                         8
1 week                          5
I don’t know yet / Not sure     5
More than 4 weeks               2
4 weeks                         2
Name: count, dtype: int64


In [641]:
print(df_clean["most_wanted_pref_to_visit"].value_counts())

most_wanted_pref_to_visit
Je n’ai pas encore d’idée précise, j’ai besoin d’y réfléchir ou de me renseigner.                                                                                                                                                                                                                                                    11
Tokyo et sa région (Kanto), Kyoto / Osaka / Nara (Kansai)                                                                                                                                                                                                                                                                             5
Tokyo et sa région (Kanto), Kyoto / Osaka / Nara (Kansai), Chūbu (Nagoya, Alpes japonaises, Kanazawa, Takayama, Mont Fuji côté Yamanashi/Shizuoka)                                                                                                                                                                            

In [642]:
def smart_split(val):
    if pd.isna(val):
        return[]
    s = str(val)
    parts = re.split(r',(?![^()]*\))', s)
    parts = [p.strip() for p in parts if p.strip()]
    return parts

In [643]:
regions_list = df_clean["most_wanted_pref_to_visit"].apply(smart_split)

In [644]:
MAX_CHOICES = 5

def list_to_fixed_cols(lst, k=MAX_CHOICES):
    lst = (lst + [np.nan]*k) [:k]
    return pd.Series(lst, index=[f"pref_{i+1}" for i in range(k)])

df_prefs = regions_list.apply(list_to_fixed_cols)
df_clean = pd.concat([df_clean, df_prefs], axis=1)


In [645]:
display(df_clean.head())

,Horodateur,nationality,country,age_group,family_situation,household_income_in_€,travel_frequency,been_to_Japan,Japan_vac_duration,most_wanted_pref_to_visit,...,trip_prep,booking_trip_channel,most_influencial_reason_to_choose_dest,alt_dest_most_difficulties,recomendation_to_improve_attractiveness,pref_1,pref_2,pref_3,pref_4,pref_5
0,05/10/2025 13:09:39,France,France,35-44,Married_no_kids,1500-1999,Every 2–3 years,"Yes, several times",More than 4 weeks,"Kyoto / Osaka / Nara (Kansai), Région du Tohok...",...,Réseaux sociaux / influenceurs,Site officiel de compagnies aériennes ou hôtels,Explorer le patrimoine culturel et historique ...,"Barrière de la langue, Difficultés avec les tr...",Le Japon est parfait tel qu'il est,Kyoto / Osaka / Nara (Kansai),"Région du Tohoku (ex. Yamagata, Sendai)","Shikoku (île du pèlerinage des 88 temples, Mat...",NaN,NaN
1,06/10/2025 13:30:50,France,France,45-54,Relationship_with_kids,1500-1999,Every 2–3 years,"No, but I would like to go",2 weeks,"Je n’ai pas encore d’idée précise, j’ai besoin...",...,"Guides papier (Lonely Planet, Routard…)","Agence en ligne (ex. Expedia, Booking.com)",Explorer le patrimoine culturel et historique ...,"Coût de la vie (hébergement, nourriture, activ...","Déjà très attractif pour moi, juste une questi...",Je n’ai pas encore d’idée précise,j’ai besoin d’y réfléchir ou de me renseigner.,NaN,NaN,NaN
2,06/10/2025 17:20:05,France,France,35-44,Single,1500-1999,Once a year,"No, but I would like to go",3 weeks,"Je n’ai pas encore d’idée précise, j’ai besoin...",...,"Sites spécialisés (Voyageurs du Monde, Comptoi...","Plateformes collaboratives (Airbnb, etc.)",Vivre une expérience unique ou dépaysante,"Barrière de la langue, Problèmes liés à la loc...",son prix,Je n’ai pas encore d’idée précise,j’ai besoin d’y réfléchir ou de me renseigner.,NaN,NaN,NaN
3,06/10/2025 19:47:27,France,France,45-54,Single,2000-2499,Every 2–3 years,"No, but I would like to go",1 week,"Je n’ai pas encore d’idée précise, j’ai besoin...",...,"Guides papier (Lonely Planet, Routard…)","Agence en ligne (ex. Expedia, Booking.com)",Vivre une expérience unique ou dépaysante,combiner toutes les activités avec les lieux d...,un guide chatgpt,Je n’ai pas encore d’idée précise,j’ai besoin d’y réfléchir ou de me renseigner.,NaN,NaN,NaN
4,06/10/2025 20:56:00,France,France,45-54,Married_no_kids,2500-2999,Every 2–3 years,"No, but I would like to go",3 weeks,Kyoto / Osaka / Nara (Kansai),...,"Sites spécialisés (Voyageurs du Monde, Comptoi...","Agence en ligne (ex. Expedia, Booking.com)",Découvrir la nature et les paysages,Barrière de la langue,Moins cher,Kyoto / Osaka / Nara (Kansai),NaN,NaN,NaN,NaN


In [646]:
clean_most_wanted_pref_to_visit = {
    
    "tokyo et sa region (kanto)": "Kanto",
    "tokyo and its region (kanto)": "Kanto",

    "kyoto / osaka / nara (kansai)": "Kansai",

    "region du tohoku (ex. yamagata, sendai)": "Tohoku",
    "tohoku region (e.g. yamagata, sendai)": "Tohoku",

    "chūgoku (hiroshima, miyajima, okayama, matsue)": "Chūgoku",

    "shikoku (ile du pelerinage des 88 temples, matsuyama, iya valley)": "Shikoku",
    "shikoku (88 temple pilgrimage island, matsuyama, iya valley)": "Shikoku",

    "chūbu (nagoya, alpes japonaises, kanazawa, takayama, mont fuji côte yamanashi/shizuoka)": "Chūbu",
    "chubu (nagoya, japanese alps, kanazawa, takayama, mt. fuji – yamanashi/shizuoka side)": "Chūbu",

    "je n’ai pas encore d’idee precise, j’ai besoin d’y reflechir ou de me renseigner.": "Unknown",
    "j’ai besoin d’y reflechir ou de me renseigner.": None,
    "je n’ai pas encore d’idee precise": "Unknown",
    "i don’t have a clear idea yet / i need to think or find out more": "Unknown",
    }

cols = ["pref_1", "pref_2", "pref_3", "pref_4", "pref_5"]
df_clean[cols] = (df_clean[cols]
                            .applymap(normalize_text)
                            .applymap(lambda x: clean_most_wanted_pref_to_visit.get(x, x)))



C:\Users\j-bal\AppData\Local\Temp\ipykernel_21944\3613292185.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(normalize_text)
C:\Users\j-bal\AppData\Local\Temp\ipykernel_21944\3613292185.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(lambda x: clean_most_wanted_pref_to_visit.get(x, x)))


In [647]:
print(df_clean["pref_1"].value_counts())
print(df_clean["pref_2"].value_counts())
print(df_clean["pref_3"].value_counts())
print(df_clean["pref_4"].value_counts())
print(df_clean["pref_5"].value_counts())

pref_1
Kanto       20
Unknown     12
Kansai       6
hokkaido     1
Name: count, dtype: int64
pref_2
Kansai      18
Tohoku       3
hokkaido     1
Chūgoku      1
okinawa      1
Unknown      1
Name: count, dtype: int64
pref_3
hokkaido    5
Chūbu       4
Shikoku     3
okinawa     3
Unknown     2
Chūgoku     1
Name: count, dtype: int64
pref_4
okinawa    4
Name: count, dtype: int64
pref_5
Chūbu      1
Tohoku     1
Chūgoku    1
Shikoku    1
Name: count, dtype: int64


In [648]:
print(df_clean["rating_interest_culture_and_history"].value_counts())

rating_interest_culture_and_history
Très important          15
Essentiel                9
Assez important          9
Peu important            3
Essential                2
Moderately important     1
Name: count, dtype: int64


In [649]:
clean_rating_japan = {
    "Pas du tout important": "Not important at all",
    "Peu important": "Slightly important",
    "Assez important": "Moderately important",
    "Très important": "Very important",
    "Essentiel": "Essential",

}
rating_cols = ['rating_interest_culture_and_history', 'rating_interest_food',
       'rating_interest_nature_hiking', 'rating_interest_shopping_and_techno',
       'rating_interest_events_and_festivals', 'rating_interest_wellness',
       'rating_interest_theme_park']
df_clean[rating_cols] = (df_clean[rating_cols]
                            .applymap(lambda x: clean_rating_japan.get(x, x)))



C:\Users\j-bal\AppData\Local\Temp\ipykernel_21944\1417326439.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(lambda x: clean_rating_japan.get(x, x)))


In [650]:
print(df_clean["rating_interest_culture_and_history"].value_counts())
print(df_clean["rating_interest_food"].value_counts())
print(df_clean["rating_interest_nature_hiking"].value_counts())
print(df_clean["rating_interest_shopping_and_techno"].value_counts())
print(df_clean["rating_interest_events_and_festivals"].value_counts())
print(df_clean["rating_interest_wellness"].value_counts())
print(df_clean["rating_interest_theme_park"].value_counts())

rating_interest_culture_and_history
Very important          15
Essential               11
Moderately important    10
Slightly important       3
Name: count, dtype: int64
rating_interest_food
Essential               18
Very important          10
Moderately important     8
Slightly important       3
Name: count, dtype: int64
rating_interest_nature_hiking
Essential               15
Very important          12
Moderately important     9
Slightly important       2
Not important at all     1
Name: count, dtype: int64
rating_interest_shopping_and_techno
Slightly important      13
Moderately important    11
Not important at all     8
Essential                4
Very important           3
Name: count, dtype: int64
rating_interest_events_and_festivals
Moderately important    18
Slightly important       9
Not important at all     5
Very important           5
Essential                2
Name: count, dtype: int64
rating_interest_wellness
Moderately important    13
Very important          10
Slightly i

In [651]:
print(df_clean["Japan_budget_per_week"].value_counts())

Japan_budget_per_week
500 – 1 000 €                           8
1 000 – 1 500 €                         8
Je ne sais pas / Pas assez renseigné    7
1 500 – 2 500 €                         7
Plus de 2 500 €                         4
Moins de 500 €                          2
$550 – $1,100 (~€500 – €1,000)          1
Less than $550 (~€500)                  1
More than $2,750 (~€2,500+)             1
Name: count, dtype: int64


In [652]:
clean_budget_japan = {
    "Moins de 500 €": "Less than 500",
    "Less than $550 (~€500)": "Less than 500",

    "500 – 1 000 €": "500-1000",
    "$550 – $1,100 (~€500 – €1,000)": "500-1000",

    "1 000 – 1 500 €": "1000-1500",
    "$1,100 – $1,650 (~€1,000 – €1,500)": "1000-1500",

    "1 500 – 2 500 €": "1500-2500",
    "$1,650 – $2,750 (~€1,500 – €2,500)": "1500-2500",

    "Plus de 2 500 €": "More than 2500",
    "More than $2,750 (~€2,500+)": "More than 2500",

    "Je ne sais pas / Pas assez renseigné": "Unknown",
    "I don’t know / Not sure yet": "Unknown",

}
df_clean["Japan_budget_per_week"] = (df_clean["Japan_budget_per_week"]
                            .map(clean_budget_japan)
                            .fillna(df_clean["Japan_budget_per_week"]))


In [654]:
print(df_clean["Japan_budget_per_week"].value_counts())

Japan_budget_per_week
500-1000          9
1000-1500         8
Unknown           7
1500-2500         7
More than 2500    5
Less than 500     3
Name: count, dtype: int64


In [656]:
print(df_clean["Japan_prefered_accomodation"].value_counts())

Japan_prefered_accomodation
Hôtel classique (3–4 étoiles)             15
Ryokan (auberge traditionnelle)           12
Airbnb / logement chez l’habitant          5
Hôtel haut de gamme / luxe (5 étoiles)     2
Capsule hôtel                              1
hostel/ auberge de jeunesse                1
Airbnb / homestay                          1
Standard hotel (3–4 stars)                 1
Any                                        1
Name: count, dtype: int64


In [ ]:
print(df_clean["household_income_in_€"].value_counts())

clean_income = {
    
    "moins de 1 500 ": "1500 and less",
    "less than 1,700 (~1,500)": "1500 and less",

    "prefere ne pas repondre": "Unknown",
    "prefer not to say": "Unknown",
    }

df_clean["household_income_in_€"] = (df_clean["household_income_in_€"]
                            .map(normalize_text)
                            .map(clean_income)
                            .fillna(df_clean["household_income_in_€"]))

print(df_clean["household_income_in_€"].value_counts())

household_income_in_€
1500-1999        11
3000-3999         7
2000-2499         5
2500-2999         5
5000–5999         5
Unknown           4
7000 and more     3
1500 and less     3
4000–4999         2
6000–6999         1
Name: count, dtype: int64
household_income_in_€
1500-1999        11
3000-3999         7
2000-2499         5
2500-2999         5
5000–5999         5
Unknown           4
7000 and more     3
1500 and less     3
4000–4999         2
6000–6999         1
Name: count, dtype: int64


In [655]:
df_clean.columns

Index(['Horodateur', 'nationality', 'country', 'age_group', 'family_situation',
       'household_income_in_€', 'travel_frequency', 'been_to_Japan',
       'Japan_vac_duration', 'most_wanted_pref_to_visit',
       'rating_interest_culture_and_history', 'rating_interest_food',
       'rating_interest_nature_hiking', 'rating_interest_shopping_and_techno',
       'rating_interest_events_and_festivals', 'rating_interest_wellness',
       'rating_interest_theme_park', 'Japan_budget_per_week',
       'Japan_prefered_accomodation', 'Japan_most_difficulties',
       'alternative_destination', 'alt_dest_main_reason',
       'alt_dest_prefered_accomodation', 'alt_dest_budget_per_week',
       'alt_dest_transportation', 'trip_prep', 'booking_trip_channel',
       'most_influencial_reason_to_choose_dest', 'alt_dest_most_difficulties',
       'recomendation_to_improve_attractiveness', 'pref_1', 'pref_2', 'pref_3',
       'pref_4', 'pref_5'],
      dtype='object')